<a href="https://colab.research.google.com/github/Somiya11/trial/blob/main/Finetuned_model_on_part2_dataset_17July.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.8 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.2 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from transformers import Trainer, TrainingArguments

In [ ]:
class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.questions = df['question'].tolist()
        self.queries = df['sql'].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        query = self.queries[idx]

        inputs = self.tokenizer.encode_plus(
            question,
            padding='max_length',
            truncation=True,
            max_length=256,
            return_tensors='pt'
        )
        labels = self.tokenizer.encode(
            query,
            padding='max_length',
            truncation=True,
            max_length=256,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': labels.squeeze()
        }

In [ ]:
df = pd.read_excel('NLtoSQL_dataset_part2.xls')

In [ ]:
df

,Natural_language,SQL
0,How many unique major codes are there in oltas?,SELECT DISTINCT(MAJ_HD_CD) FROM Table_oltas
1,How many unique minor codes are there in oltas?,SELECT DISTINCT(MINR_HD_CD) FROM Table_oltas
2,How many unique bank codes are there in oltas?,SELECT DISTINCT(BANK_CD) FROM Table_oltas
3,How many unique branch codes are there in oltas?,SELECT DISTINCT(BRANCH_CD) FROM Table_oltas
4,List all unique taxpayers who have deposited c...,SELECT DISTINCT(TAN_PAN) FROM Table_oltas WHER...
...,...,...
194,"List full name, age and tax paid by taxpayers ...","Select A.Full_Name, B.TAX\nfrom Table_PAN A\nl..."
195,"List full name, age and tax paid by taxpayers ...","Select A.Full_Name, B.TAX\nfrom Table_PAN A\nl..."
196,What are the maximum and minimum tax paid in I...,"SELECT MAX(TAX), MIN(TAX) FROM Table_Itr1"
197,"What are the maximum, minimum and average tax ...","SELECT MAX(TAX), MIN(TAX), AVG(TAX) FROM Table..."


# New Section

# New Section

In [ ]:
df=df.sample(frac=1)

In [ ]:
df.head()

,Natural_language,SQL
28,How much TDS advance tax was collected through...,SELECT SUM(TOT_AMT) FROM Table_oltas WHERE M...
38,How many unique Filing type are present in tab...,SELECT DISTINCT(Filing_type) FROM Table_efiling
81,Show all data in ITR4 table where PAN is not i...,SELECT * FROM Table_itr4 where PAN<>'invalid'
147,Show the maximum advance tax in oltas table,SELECT MAX(TOT_AMT) FROM Table_oltas WHERE M...
109,Show total tax paid in ITR4 table,SELECT Sum(TAX) FROM Table_itr4


In [ ]:
#df=df.head(950)

In [ ]:
df2=df.copy()

In [ ]:
df2.shape

(199, 2)

In [ ]:
df2.rename(columns={'Natural_language':'question', 'SQL':'sql'}, inplace=True)

In [ ]:
#df_new=pd.concat([df, df2])

In [ ]:
#df_new.shape

In [ ]:
df_new=df2.copy()

In [ ]:
train_data = df_new
valid_data = df_new[170:]
#test_data = df[45000:]

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('dbernsohn/t5_wikisql_SQL2en')
model = T5ForConditionalGeneration.from_pretrained('dbernsohn/t5_wikisql_SQL2en')

In [ ]:
dataset1 = SQLDataset(train_data, tokenizer)
dataset2 = SQLDataset(valid_data, tokenizer)

In [ ]:
dataset1[0]

{'input_ids': tensor([  571,   231,   332,  3592,  3245,  1104,    47,  4759,   190, 11588,
         12311,   460,  2658,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=50,
    per_device_train_batch_size=3,    #changed from 16 to 5
    per_device_eval_batch_size=3,
    learning_rate=1e-3,
    save_total_limit=1,
    fp16=True,  # Enable mixed-precision training if available
    logging_dir='./logs',
    logging_steps=10,  # changed from 500 to 10
    save_steps=500,
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset1,
    eval_dataset=dataset2,
)


In [ ]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
trainer.train()  #training 1

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,7.743300
20,0.980600
30,0.531700
40,0.328200
50,0.375800
60,0.198900
70,0.242600
80,0.090800
90,0.315000
100,0.085600


TrainOutput(global_step=3350, training_loss=0.038949629961071645, metrics={'train_runtime': 335.2713, 'train_samples_per_second': 29.677, 'train_steps_per_second': 9.992, 'total_flos': 673325462323200.0, 'train_loss': 0.038949629961071645, 'epoch': 50.0})

In [ ]:
#trainer.train()  #training 2

In [ ]:
save_path = 'trained_model_17july'
!mkdir {save_path}
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('trained_model_17july/tokenizer_config.json',
 'trained_model_17july/special_tokens_map.json',
 'trained_model_17july/spiece.model',
 'trained_model_17july/added_tokens.json')

In [ ]:
!zip -r Model_17july.zip {save_path}

  adding: trained_model_17july/ (stored 0%)
  adding: trained_model_17july/tokenizer_config.json (deflated 83%)
  adding: trained_model_17july/pytorch_model.bin (deflated 7%)
  adding: trained_model_17july/spiece.model (deflated 48%)
  adding: trained_model_17july/config.json (deflated 62%)
  adding: trained_model_17july/generation_config.json (deflated 29%)
  adding: trained_model_17july/special_tokens_map.json (deflated 86%)


In [ ]:
output_dir = './fine-tuned-model'
trainer.save_model(output_dir)

In [ ]:
results = trainer.evaluate(dataset2)
print(results)

{'eval_loss': 2.530820893298369e-06, 'eval_runtime': 0.3002, 'eval_samples_per_second': 96.599, 'eval_steps_per_second': 33.31, 'epoch': 50.0}


In [ ]:
#training 2
results = trainer.evaluate(dataset2)
print(results)

{'eval_loss': 0.013928608037531376, 'eval_runtime': 0.4193, 'eval_samples_per_second': 69.169, 'eval_steps_per_second': 23.851, 'epoch': 100.0}


In [ ]:
## new learning rate and batch size and Model


In [ ]:
results = trainer.evaluate(dataset2)
print(results)

{'eval_loss': 5.977613454888342e-06, 'eval_runtime': 0.2662, 'eval_samples_per_second': 108.941, 'eval_steps_per_second': 22.539, 'epoch': 50.0}


In [ ]:
# after increasing lenght to 256
results = trainer.evaluate(dataset2)
print(results)

{'eval_loss': 2.530820893298369e-06, 'eval_runtime': 0.2924, 'eval_samples_per_second': 99.191, 'eval_steps_per_second': 34.204, 'epoch': 50.0}



**Check the ouput of the model trained twice for 100 epochs each**

In [ ]:
input_question = "What is the average total tax paid by coporate entities in year 2023"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT AVG(TAX) FROM Table_oltas WHERE AST_YR=2023</s>


In [ ]:
input_question = "What is the minimum regular tax paid by coporate entities in oltas table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MIN(TOT_AMT) FROM Table_oltas where MAJ_HD_CD=20</s>


In [ ]:
input_question = "What is the maximum and minimum advance tax paid by coporate entities in oltas table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=100 AND MAJ_HD_CD=20</s>


In [ ]:
input_question = "Show all data in ITR5 table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT * FROM Table_itr5</s>


In [ ]:
input_question = "Show count of distinct taxpayers in ITR2 table	"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT COUNT(DISTINCT Taxpayers) FROM Table_itr2</s>


In [ ]:
input_question = "Show full name and tax paid by PAN='abc123123"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_PAN on A.Pan='abc123123'</s>


In [ ]:
input_question = "Show full name and tax paid"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_Tax on A.Pan=B.Pan</s>


In [ ]:
input_question = "Show full name, age and tax paid whose Age is 50"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_Pan where Age=50</s>


In [ ]:
input_question = "Show full name, age and tax paid whose Age between 30 and 50"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_Tax on A.Pan=B.Pan where Age Between 30 and 50</s>


In [ ]:
input_question = "How many unique sft codes are there"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(sft_cd) FROM Table_sft</s>


In [ ]:
input_question = "unique sft codes in sft table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(sft_cd) from Table_sft</s>


In [ ]:
input_question = "unique sft codes in sft table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

In [ ]:
input_question = "Show maximum tax paid by non corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TAX) FROM Table_oltas WHERE MINR_HD_CD=100 AND MAJ_HD_CD=21 AND AST_YR=21</s>


In [ ]:
input_question = "Show full name of taxpayers who were born after 1990 "
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT FULL_NAME from Table_PAN where BIRTH_YEAR>1990</s>


In [ ]:
input_question = "Show all personal information and GST paid by taxpayers"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.*, B.TAX from Table_PAN A left join Table_GST on A.Pan=B.Pan</s>


In [ ]:
input_question = "Show all personal information and GST paid by taxpayers paid in 2020"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.*, B.TAX from Table_PAN A left join Table_GST on A.Pan=B.Pan</s>


In [ ]:
input_question = "Show all data in GST table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT * FROM Table_GST</s>


In [ ]:
input_question = "Show all data in Aman table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT * FROM Table_aman</s>


In [ ]:
input_question = "Show max and min amount in table money"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Max(TOT_AMT) FROM Table_money</s>


In [ ]:
input_question = "Show maximum, minimum amount in table money"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Max(TOT_AMT) FROM Table_money</s>


In [ ]:
input_question ="What are the maximum, minimum and average tax paid in Itr1 table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TAX), MIN(TAX), AVG(TAX) FROM Table_Itr1</s>


In [ ]:
input_question ="List PAN of all unique corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(TAN_PAN) from Table_oltas where MAJ_HD_CD=20</s>


In [ ]:
input_question ="List PAN of all unique non corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(TAN_PAN) from Table_oltas where MAJ_HD_CD=21</s>


In [ ]:
input_question ="Show max self_assessment tax"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=300</s>


In [ ]:
input_question ="Show max regular tax "
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Max(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=100 AND MAJ_HD_CD=20</s>


In [ ]:
#END

** END ** **bold text**

In [ ]:
df_new.head(50)

,question,sql
51,"Show the latest filing sequence, Form_id, fili...","SELECT FILING_SEQ_NO,FILING_TYPE,PAN_NO,FORM_I..."
137,show all unique non corporate entities in olta...,SELECT DISTINCT(TAN_PAN) from Table_oltas wher...
67,Show all data in Charity table,SELECT * FROM Table_charity
90,Show minimum TAX in ITR1 table,SELECT Min(TAX) FROM Table_itr1
80,Show all data in ITR3 table where PAN is not i...,SELECT * FROM Table_itr3 where PAN<>'invalid'
1,How many unique minor codes are there in oltas?,SELECT DISTINCT(MINR_HD_CD) FROM Table_oltas
2,How many unique bank codes are there in oltas?,SELECT DISTINCT(BANK_CD) FROM Table_oltas
158,What is the average self_assessment tax in olt...,SELECT AVG(TOT_AMT) FROM Table_oltas WHERE M...
171,Show full name of taxpayers and tax paid in Itr3,"Select A.Full_Name, B.TAX\nfrom Table_PAN A\nl..."
109,Show total tax paid in ITR4 table,SELECT Sum(TAX) FROM Table_itr4


In [ ]:
#generation from newly trained

In [ ]:
input_question = "What is the average total tax paid by coporate entities in year 2023"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT AVG(TOT_AMT) FROM Table_oltas WHERE AST_YR=2023</s>


In [ ]:
input_question = "What is the minimum regular tax paid by coporate entities in oltas table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MIN(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=400</s>


In [ ]:
input_question = "minimum selfassessment tax paid by non-coporate entities in oltas table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MIN(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=300 AND MAJ_HD_CD=21</s>


In [ ]:
input_question = "show tax amount paid by coporate entities in oltas table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT TAN_PAN from Table_oltas where MAJ_HD_CD=20</s>


In [ ]:
input_question = "show advance tax amount paid by coporate entities in oltas table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT AVG(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=100</s>


In [ ]:
input_question = "List all details filed in ITR1 by PANS ABCD123456"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=256,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=256,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT B.PAN_NO, B.ASSESSMENT_YEAR, A.* from ( SELECT FILING_SEQ_NO,FILING_TYPE,PAN_NO,FORM_ID,ASSESSMENT_YEAR,FILING_DATE FROM Table_efiling WHERE FORM_ID in ('ITR-5','ITR-3','ITR-2','ITR-1','ITR-4','ITR-6','ITR-7') AND FILING_TYPE IN ('O','R','D','U') AND ASSESSMENT_YEAR IN '2020' AND PAN_NO IN ('ABCD123456', 'EFD', 'EFI123456', 'EFD') qualify row_number() over (partition by pan_no, assessment_year order by filing_Seq_no DESC) = 1) B left join Table_Itr A on B.FILING_SEQ_NO</s>


In [ ]:
df_new

,question,sql
122,Show average TAX in ITR3 table in year 2022,SELECT AVG(TAX) FROM Table_itr3 where year=2022
151,Show the maximum regular tax in oltas table,SELECT MAX(TOT_AMT) FROM Table_oltas WHERE M...
83,Show all data in ITR2 table where PAN is valid,SELECT * FROM Table_itr2 where PAN='valid'
25,How much TDS was collected through Non Corpora...,SELECT SUM(TOT_AMT) FROM Table_oltas WHERE M...
125,who are the corporate entities in oltas table,SELECT TAN_PAN from Table_oltas where MAJ_HD_C...
...,...,...
13,How much total transaction was done before 2020,SELECT SUM(TOT_AMT) FROM Table_oltas \nWHERE A...
117,Show all unique ITDREIN in SFT table where Ret...,SELECT DISTINCT(sft_cd) from Table_SFT where R...
31,How much SAT was collected through Corporate e...,SELECT SUM(TOT_AMT) FROM Table_oltas WHERE M...
62,List all details filed in ITR1 by PANS ABCD123...,"SELECT B.PAN_NO, B.ASSESSMENT_YEAR, A.* from \..."


In [ ]:
input_question = "List all corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT TAN_PAN from Table_oltas where MAJ_HD_CD=20</s>


In [ ]:
input_question = "List all non corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT TAN_PAN from Table_oltas where MAJ_HD_CD=21</s>


In [ ]:
input_question = "Show unique itr forms"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(itr forms) from Table_itr</s>


In [ ]:
input_question = "How much TDS was collected from corporate entities."
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=7,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT SUM(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=200 AND MAJ_HD_CD=20</s>


In [ ]:
input_question = "Show unique SFT"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(sft_cd) from Table_SFT</s>


In [ ]:
Check the ouput of the model trained twice for 100 epochs each

 **Output of model after changing to 256 and training on le-3 learning rate with 3 sample**

In [ ]:
input_question = "Latest tax details by PANS 123ABCD456"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT B.PAN_NO, B.ASSESSMENT_YEAR, A.* from ( SELECT FILING_SEQ_NO,FILING_TYPE,PAN_NO,FORM_ID,ASSESSMENT_YEAR,FILING_DATE FROM Table_efiling WHERE FORM_ID in ('ITR-5','ITR-3','ITR-2','ITR-1','ITR-4','ITR-6','ITR-7') AND FILING_TYPE IN ('O','R','D','U') AND ASSESSMENT_YEAR=206' AND PAN_NO IN ('ABCD456' qualify row_number() over (partition by pan_no, assessment_year order by filing_Seq_no DESC) = 1</s>


In [ ]:
input_question = "Show unique SFT codes"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(sft_cd) from Table_SFT</s>


In [ ]:
input_question = "Show full name, age and tax paid by taxpayers in Itr1"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_Itr1 on A.Pan=B.Pan</s>


In [ ]:
input_question = "List the maximum and minimum of tax paid in ITR3"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TAX), MIN(TAX) FROM Table_itr3</s>


In [ ]:
input_question = "Show average selfassessment tax"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT AVG(TAX) FROM Table_oltas WHERE MINR_HD_CD=300</s>


In [ ]:
input_question = "Show average advance tax by corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT AVG(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=100 AND MAJ_HD_CD=20</s>


In [ ]:
input_question = "Show total regular tax by corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Sum(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=400 AND MAJ_HD_CD=20</s>


In [ ]:
input_question = "Show maximum, minimum, total regular tax by corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Max(TAX), MIN(TAX), AVG(TAX) FROM Table_oltas WHERE MINR_HD_CD=400 AND MAJ_HD_CD=20</s>


In [ ]:
input_question = "Show maximum, minimum, total regular tax by non-corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Max(TAX), MIN(TAX), AVG(TAX) FROM Table_oltas WHERE MINR_HD_CD=400 AND MAJ_HD_CD=21 AND AST_YR=21</s>


In [ ]:
input_question = "Show count of non-corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=320,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=320,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT COUNT(DISTINCT TAN_PAN)) from Table_oltas where MAJ_HD_CD=21</s>


In [ ]:
input_question = "average tax paid by coporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT AVG(TAX) FROM Table_oltas WHERE MINR_HD_CD=100 AND MAJ_HD_CD=20</s>


In [ ]:
input_question = "Full name of taxpayers who paid advance tax and GST paid in year 2020 and month June"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=256,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=256,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.GST C.TOT_AMT from Table_PAN A left join Table_GST B on A.Pan=B.Pan where Year=2020 and Month='June' and MINR_HD_CD=100</s>


In [ ]:
input_question = "Show Full name of taxpayers who paid advance tax and GST paid in year 2020 and month June"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=256,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=256,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.GST C.TOT_AMT from Table_PAN A left join Table_GST B on A.Pan=B.Pan where Year=2020 and Month='June' and MINR_HD_CD=100</s>


In [ ]:
input_question = "Show latest filing sequence form_id of PAN ABCD12345 in AY 2021 "
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=256,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=256,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT FILING_SEQ_NO FROM Table_efiling WHERE FORM_ID in ('ITR-5','ITR-3','ITR-2','ITR-1','ITR-4','ITR-6','ITR-7') AND FILING_TYPE IN ('O','R','D','U') AND ASSESSMENT_YEAR=2021 AND PAN_NO='ABCD12345' qualify row_number() over (partition by pan_no, assessment_year order by filing_Seq_no DESC) = 1</s>
